# Libraries

In [1]:
import gradio as gr
import pandas as pd
import numpy as np
import json
import io 
import sys
from PIL import Image
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.faiss import FAISS

c:\Users\bpadmin\anaconda3\envs\tf_tr_recommender\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# vector db creation

In [2]:
stock_data_excel_file = pd.ExcelFile('../../data/phase_2/stock_data.xlsx')

In [3]:
data_path = '../../data/phase_2/price_data_v2.xlsx'

data = pd.read_excel(data_path, index_col= False)

In [4]:
price_symbols = list(data.SECURITYCODE.unique())

In [5]:
# encoded_stock_data_path = '../data/sl20.xlsx'
# stock_data = pd.read_csv(encoded_stock_data_path)
stock_data = pd.read_excel(stock_data_excel_file, 'phase 1 symbols reduced')
stock_data.head(2)


,GICS Category,Symbol,Name,Business_Summary
0,Information Technology,HBS,hSenid Business Solutions PLC,development of human capital management (HCM) ...
1,Consumer Discretionary,TYRE,KELANI TYRES PLC,"importation and sale of tires. In addition, th..."


In [6]:
stock_data.Symbol.nunique()

81

In [7]:
stock_data = stock_data[stock_data.Symbol.isin(price_symbols)].copy()
stock_data.Symbol.nunique()

73

In [8]:
stock_data.isna().any()

GICS Category       False
Symbol              False
Name                False
Business_Summary    False
dtype: bool

In [9]:
# stock_data = stock_data.fillna('unknown')
stock_data['Symbol'] = stock_data['Symbol'].str.replace(r"(","")
stock_data['Symbol'] = stock_data['Symbol'].str.replace(r")","")


C:\Users\naradaw\AppData\Local\Temp\ipykernel_19016\423448387.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  stock_data['Symbol'] = stock_data['Symbol'].str.replace(r"(","")
C:\Users\naradaw\AppData\Local\Temp\ipykernel_19016\423448387.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  stock_data['Symbol'] = stock_data['Symbol'].str.replace(r")","")


In [10]:
stock_data.Name

0     hSenid Business Solutions PLC
1                  KELANI TYRES PLC
2             ABANS ELECTRICALS PLC
3           DANKOTUWA PORCELAIN PLC
4                 HAYLEYS FIBRE PLC
                  ...              
76                 LOLC FINANCE PLC
77                   MELSTACORP PLC
78           NATIONS TRUST BANK PLC
79         ROYAL CERAMICS LANKA PLC
80                 SAMPATH BANK PLC
Name: Name, Length: 73, dtype: object

In [11]:
embeddings = HuggingFaceEmbeddings()
stock_meta_data = [{'symbol': stock_data.iloc[idx]['Symbol'], 'name' : stock_data.iloc[idx]['Name']} for idx in range(stock_data.shape[0])]
stock_meta_data
originals = [ stock_data.iloc[idx]['Name'] + ' ' + stock_data.iloc[idx]['Business_Summary'] for idx in range(stock_data.shape[0])]
originals[:5]


["hSenid Business Solutions PLC development of human capital management (HCM) software products and related services. The Company's offerings include a comprehensive human resource information system (HRIS) solution known as the PeoplesHR platform as well as additional solutions such as PeoplesHR Outsourcing and Tracking Solutions.",
 'KELANI TYRES PLC importation and sale of tires. In addition, the Company is engaged in holding investments in its subsidiary and joint venture as well as earning rental income from investment property.',
 'ABANS ELECTRICALS PLC manufacturing, assembling and installation of selected consumer durable products, provision of repair and maintenance services for a wider range of household electric and electronic appliances and installation of roof top Solar photovoltaic (PV) Systems.',
 'DANKOTUWA PORCELAIN PLC manufacturing and marketing of porcelain tableware, targeted to export and domestic markets.',
 'HAYLEYS FIBRE PLC manufacture and export of coir fibre

In [15]:
stock_data.loc[stock_data['Symbol'] == 'TAJ']

,GICS Category,Symbol,Name,Business_Summary
8,Consumer Discretionary,TAJ,TAL LANKA HOTELS PLC,luxary hotels and resorts.


In [16]:
from langchain_community.vectorstores import FAISS
vectors = embeddings.embed_documents(originals)
'''Tuple[str, List[float]]'''

stock_data_embeddings = [(originals[idx],vectors[idx]) for idx in range(len(vectors))]
db = FAISS.from_embeddings(text_embeddings = stock_data_embeddings, embedding = embeddings, metadatas= stock_meta_data)
db.save_local("../../vector_db/phase3_symbols")
